### 实施 FunkSVD

在此 notebook 中，我们将自己编写实施 FunkSVD 的函数，它会遵守你在上个视频中看到的步骤。如果你认为你还没有准备好自己完成这个任务，可以跳到下个视频，观看我是如何完成这些步骤的。

我们将用之前使用的数据子集检测我们的算法。首先请运行以下单元格。

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import sparse
import svd_tests as t
%matplotlib inline

# Read in the datasets
movies = pd.read_csv('movies_clean.csv')
reviews = pd.read_csv('reviews_clean.csv')

del movies['Unnamed: 0']
del reviews['Unnamed: 0']

# Create user-by-item matrix
user_items = reviews[['user_id', 'movie_id', 'rating', 'timestamp']]
user_by_movie = user_items.groupby(['user_id', 'movie_id'])['rating'].max().unstack()

# Create data subset
user_movie_subset = user_by_movie[[73486, 75314,  68646, 99685]].dropna(axis=0)
ratings_mat = np.matrix(user_movie_subset)
display(user_movie_subset)
display(ratings_mat)

movie_id,73486,75314,68646,99685
user_id,,,,
265,10.0,10.0,10.0,10.0
1023,10.0,4.0,9.0,10.0
1683,8.0,9.0,10.0,5.0
6571,9.0,8.0,10.0,10.0
11639,10.0,5.0,9.0,9.0
13006,6.0,4.0,10.0,6.0
14076,9.0,8.0,10.0,9.0
14725,10.0,5.0,9.0,8.0
23548,7.0,8.0,10.0,8.0


matrix([[10., 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

`1.` 你将使用 **user_movie_subset** 矩阵演示你的 FunkSVD 算法将收敛。在以下单元格中，请参考注释和文档字符串编写你自己的 FunkSVD 函数。你也可以自己完成这些函数，不参考注释部分。你可以随意增减函数代码，使你的代码能正常运行。 

**注意：**此版矩阵分解没有 ∑ 矩阵。

In [3]:
def FunkSVD(ratings_mat, latent_features=4, learning_rate=0.0001, iters=100):
    '''
    This function performs matrix factorization using a basic form of FunkSVD with no regularization
    
    INPUT:
    ratings_mat - (numpy array) a matrix with users as rows, movies as columns, and ratings as values
    latent_features - (int) the number of latent features used
    learning_rate - (float) the learning rate 
    iters - (int) the number of iterations
    
    OUTPUT:
    user_mat - (numpy array) a user by latent feature matrix
    movie_mat - (numpy array) a latent feature by movie matrix
    '''
    
    # Set up useful values to be used through the rest of the function
    n_users = ratings_mat.shape[0]
    n_movies = ratings_mat.shape[1]
    num_ratings = np.count_nonzero(~np.isnan(ratings_mat))
    
    # initialize the user and movie matrices with random values
    user_mat = np.random.rand(n_users, latent_features)
    movie_mat = np.random.rand(latent_features, n_movies)
    
    # initialize sse at 0 for first iteration
    sse_accum = 0
    
    # header for running results
    print("Optimizaiton Statistics")
    print("Iterations | Mean Squared Error ")
    
    # for each iteration
    for iteration in range(iters):

        # update our sse
        old_sse = sse_accum
        sse_accum = 0
        
        # For each user-movie pair
        for i in range(n_users):
            for j in range(n_movies):
                
                # if the rating exists
                if ratings_mat[i, j] > 0:
                    
                    # compute the error as the actual minus the dot product of the user and movie latent features
                    diff = ratings_mat[i, j] - np.dot(user_mat[i, :], movie_mat[:, j])
                    
                    # Keep track of the sum of squared errors for the matrix
                    sse_accum += diff**2
                    
                    # update the values in each matrix in the direction of the gradient
                    for k in range(latent_features):
                        user_mat[i, k] += learning_rate * (2*diff*movie_mat[k, j])
                        movie_mat[k, j] += learning_rate * (2*diff*user_mat[i, k])

        # print results for iteration
        print("%d \t\t %f" % (iteration+1, sse_accum / num_ratings))
        
    return user_mat, movie_mat 

`2.` 请在 **user_movie_subset** 数据集上尝试你的函数。先将潜在特征设为 4 个，学习速率设为 0.005，并迭代 10 次。在计算生成的 U 和 V 矩阵的点积时，生成的 **user_movie** 矩阵与原始数据子集有何区别？

In [4]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=10)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 41.588316
2 		 13.623367
3 		 3.828497
4 		 2.723636
5 		 2.599153
6 		 2.542822
7 		 2.492961
8 		 2.439315
9 		 2.378736
10 		 2.310014


In [5]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.93154592  8.93777794 10.53443495 10.25785382]
 [ 8.83083464  7.39389796  9.18725287  8.59348186]
 [ 7.81818871  6.70578964  8.5732434   7.66540748]
 [ 9.63541598  8.42103676  9.90129442  9.56385414]
 [ 8.43361051  7.49457168  8.75162418  8.75352971]
 [ 6.60226822  5.86447877  7.85927611  6.52943459]
 [ 9.2446986   7.87799367  9.80108915  9.16333922]
 [ 8.17434221  7.08630532  8.57243243  8.3088652 ]
 [ 8.41730722  7.56785275  8.86383132  8.486757  ]
 [ 7.75224109  6.74625909  8.22431027  7.48507698]
 [ 8.951814    7.92721575  9.25085843  8.8035844 ]
 [ 9.69529947  8.59089556  9.88830058  9.54624691]
 [ 9.12608208  8.12712901  9.88839415  9.19011127]
 [ 6.55257785  5.80760033  6.96430172  6.61733479]
 [ 9.68826334  8.43606391 10.24604816  9.79045929]
 [ 9.79353793  8.49926093 10.12468893  9.79739749]
 [ 8.52663084  7.47865832  8.18293175  8.47447318]
 [ 7.48765842  6.52085282  6.26386865  7.6508262 ]
 [ 8.21244648  7.31121564  8.87718963  8.40782522]
 [ 8.10499834  6.78503828  8.98

**请在此处填写你的结论。**

`3.` 再在 **user_movie_subset** 数据集上尝试你的函数。这次潜在特征为 4 个，学习速率设为 0.005。但是，迭代次数提高到 250 次。在计算生成的 U 和 V 矩阵的点积时，生成的 **user_movie** 矩阵与原始数据子集有何区别？迭代 250 次后，误差会怎样？

In [6]:
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 45.067728
2 		 16.291262
3 		 4.559872
4 		 2.814612
5 		 2.576711
6 		 2.504930
7 		 2.456176
8 		 2.406975
9 		 2.353048
10 		 2.293851
11 		 2.229588
12 		 2.160692
13 		 2.087764
14 		 2.011560
15 		 1.932970
16 		 1.852957
17 		 1.772487
18 		 1.692444
19 		 1.613571
20 		 1.536422
21 		 1.461358
22 		 1.388566
23 		 1.318110
24 		 1.249990
25 		 1.184201
26 		 1.120780
27 		 1.059832
28 		 1.001534
29 		 0.946120
30 		 0.893852
31 		 0.844984
32 		 0.799725
33 		 0.758209
34 		 0.720481
35 		 0.686483
36 		 0.656065
37 		 0.628996
38 		 0.604986
39 		 0.583703
40 		 0.564798
41 		 0.547919
42 		 0.532725
43 		 0.518898
44 		 0.506144
45 		 0.494198
46 		 0.482824
47 		 0.471812
48 		 0.460979
49 		 0.450162
50 		 0.439221
51 		 0.428034
52 		 0.416497
53 		 0.404522
54 		 0.392039
55 		 0.378994
56 		 0.365350
57 		 0.351092
58 		 0.336220
59 		 0.320759
60 		 0.304754
61 		 0.288270
62 		 0.271395
63 		 0.254236
64 		

In [7]:
#Compare the predicted and actual results
print(np.dot(user_mat, movie_mat))
print(ratings_mat)

[[ 9.99999689  9.99999868  9.9999993  10.00000302]
 [ 9.99999785  3.99999911  8.99999956 10.00000211]
 [ 8.00000193  9.00000095 10.00000053  4.99999808]
 [ 9.0000082   8.00000362 10.00000196  9.99999198]
 [10.00000568  5.0000024   9.00000126  8.99999443]
 [ 5.99999061  3.99999583  9.99999772  6.00000914]
 [ 8.99999944  7.9999997   9.99999983  9.00000055]
 [10.00000185  5.00000082  9.00000042  7.99999815]
 [ 7.00000787  8.00000338 10.00000178  7.99999229]
 [ 8.99999388  4.99999725  8.99999848  7.00000593]
 [ 8.9999987   7.99999934  9.99999963  8.00000127]
 [ 8.99999785  9.99999899  9.99999943  9.00000208]
 [10.00000438  9.00000188 10.000001    7.99999571]
 [ 4.99999606  7.99999817  4.99999898  8.00000385]
 [10.00000653  8.00000283 10.00000148  9.99999356]
 [ 8.99999678  8.99999849  9.99999912 10.00000309]
 [ 8.99999029  7.99999573  7.99999769  8.00000946]
 [10.00000007  8.00000007  1.00000005  9.99999992]
 [ 5.0000004   6.00000026 10.00000016  9.99999958]
 [ 8.00000303  7.0000014  10.00

**请在此处填写你的结论。**

上次在此矩阵里放入 **np.nan** 值时，python 中的整个 SVD 算法崩溃了。我们看看使用你的 FunkSVD 函数是否依然会这样。在以下单元格中，我在你的 numpy 数组的第一个单元格中放入了一个 NaN 值。  

`4.` 假设潜在特征有 4 个，学习速率为 0.005，并且迭代 250 次。你能够运行你的 SVD 并且不崩溃吗（内置的 python 方法会崩溃）？你获得了 NaN 值的预测值吗？缺失值的预测是多少？请在以下单元格中回答这些问题。

In [8]:
# Here we are placing a nan into our original subset matrix
ratings_mat[0, 0] = np.nan
ratings_mat

matrix([[nan, 10., 10., 10.],
        [10.,  4.,  9., 10.],
        [ 8.,  9., 10.,  5.],
        [ 9.,  8., 10., 10.],
        [10.,  5.,  9.,  9.],
        [ 6.,  4., 10.,  6.],
        [ 9.,  8., 10.,  9.],
        [10.,  5.,  9.,  8.],
        [ 7.,  8., 10.,  8.],
        [ 9.,  5.,  9.,  7.],
        [ 9.,  8., 10.,  8.],
        [ 9., 10., 10.,  9.],
        [10.,  9., 10.,  8.],
        [ 5.,  8.,  5.,  8.],
        [10.,  8., 10., 10.],
        [ 9.,  9., 10., 10.],
        [ 9.,  8.,  8.,  8.],
        [10.,  8.,  1., 10.],
        [ 5.,  6., 10., 10.],
        [ 8.,  7., 10.,  7.]])

In [9]:
# run SVD on the matrix with the missing value
user_mat, movie_mat = FunkSVD(ratings_mat, latent_features=4, learning_rate=0.005, iters=250)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 41.402850
2 		 13.869042
3 		 3.911018
4 		 2.798471
5 		 2.687558
6 		 2.649450
7 		 2.619525
8 		 2.588557
9 		 2.554863
10 		 2.518175
11 		 2.478434
12 		 2.435608
13 		 2.389673
14 		 2.340625
15 		 2.288468
16 		 2.233194
17 		 2.174759
18 		 2.113050
19 		 2.047857
20 		 1.978864
21 		 1.905646
22 		 1.827702
23 		 1.744515
24 		 1.655643
25 		 1.560842
26 		 1.460197
27 		 1.354251
28 		 1.244097
29 		 1.131396
30 		 1.018309
31 		 0.907309
32 		 0.800922
33 		 0.701417
34 		 0.610544
35 		 0.529350
36 		 0.458150
37 		 0.396601
38 		 0.343885
39 		 0.298914
40 		 0.260513
41 		 0.227567
42 		 0.199101
43 		 0.174315
44 		 0.152579
45 		 0.133413
46 		 0.116455
47 		 0.101429
48 		 0.088123
49 		 0.076363
50 		 0.065999
51 		 0.056901
52 		 0.048947
53 		 0.042021
54 		 0.036017
55 		 0.030830
56 		 0.026367
57 		 0.022537
58 		 0.019259
59 		 0.016461
60 		 0.014075
61 		 0.012043
62 		 0.010314
63 		 0.008843
64 		

In [10]:
# Run this cell to see if you were able to predict for the missing value
preds = np.dot(user_mat, movie_mat)
print("The predicted value for the missing rating is {}:".format(preds[0,0]))
print()
print("The actual value for the missing rating is {}:".format(ratings_mat[0,0]))
print()
assert np.isnan(preds[0,0]) == False
print("That's right! You just predicted a rating for a user-movie pair that was never rated!")
print("But if you look in the original matrix, this was actually a value of 10. Not bad!")

The predicted value for the missing rating is 9.810521777555415:

The actual value for the missing rating is nan:

That's right! You just predicted a rating for a user-movie pair that was never rated!
But if you look in the original matrix, this was actually a value of 10. Not bad!


下面扩展到更真实的例子上。遗憾的是，依然不建议在本地机器上对整个用户-电影矩阵运行此函数。但是，我们能够查看这个示例扩展到 1000 个用户的情况。在上述部分，你使用了一个非常小的数据子集，并且没有缺失值。

`5.` 考虑到此矩阵的大小，运行时间会比较长。假设超参数如下所示：潜在特征为 4 个，学习速率为0.005，迭代次数为20。运行时间会很久，吃点东西，散散步吧。

In [11]:
# Setting up a matrix of the first 1000 users with movie ratings
first_1000_users = np.matrix(user_by_movie.head(1000))

# perform funkSVD on the matrix of the top 1000 users
user_mat, movie_mat = FunkSVD(first_1000_users, latent_features=4, learning_rate=0.005, iters=20)

Optimizaiton Statistics
Iterations | Mean Squared Error 
1 		 23.313583
2 		 10.771609
3 		 7.377985
4 		 5.679119
5 		 4.609647
6 		 3.862078
7 		 3.305858
8 		 2.875038
9 		 2.532252
10 		 2.254322
11 		 2.025820
12 		 1.835917
13 		 1.676691
14 		 1.542141
15 		 1.427612
16 		 1.329430
17 		 1.244669
18 		 1.170995
19 		 1.106540
20 		 1.049800


`6.` 现在你已经获得了每个用户-电影对的预测值，请根据你的结果回答几个问题。请在以下每个变量对应的括号里填写正确的值，并使用以下测试检测你的答案。

In [14]:
# How many actual ratings exist in first_1000_users
num_ratings = np.count_nonzero(~np.isnan(first_1000_users))
print("The number of actual ratings in the first_1000_users is {}.".format(num_ratings))
print()

# How many ratings did we make for user-movie pairs that didn't have ratings
ratings_for_missing = first_1000_users.shape[0]*first_1000_users.shape[1] - num_ratings
print("The number of ratings made for user-movie pairs that didn't have ratings is {}".format(ratings_for_missing))

The number of actual ratings in the first_1000_users is 10852.

The number of ratings made for user-movie pairs that didn't have ratings is 31234148


In [15]:
# Test your results against the solution
assert num_ratings == 10852, "Oops!  The number of actual ratings doesn't quite look right."
assert ratings_for_missing == 31234148, "Oops!  The number of movie-user pairs that you made ratings for that didn't actually have ratings doesn't look right."

# Make sure you made predictions on all the missing user-movie pairs
preds = np.dot(user_mat, movie_mat)
assert np.isnan(preds).sum() == 0
print("Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?")

Nice job!  Looks like you have predictions made for all the missing user-movie pairs! But I still have one question... How good are they?


```python

```